# **Reproducing M1+M2 results (Table 3)**

<a target="_blank" href="https://colab.research.google.com/github/blackswan-advitamaeternam/HVAE/blob/raph/paper_experiments/Table3_exp.ipynb"> <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/> </a>

## **Colab setup**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
# to avoid having the data on your drive
%cd /content

In [ ]:
!git clone https://github.com/blackswan-advitamaeternam/HVAE.git
%cd HVAE
!git checkout raph
!pip install -r requirements.txt

## **Imports**

In [ ]:
import sys, os
# To ensure the custom package is found
path_to_repo = "/content/HVAE"
if path_to_repo not in sys.path:
    sys.path.append(path_to_repo)

In [ ]:
import numpy as np
import pandas as pd 
from tqdm.auto import tqdm
from sklearn.metrics import accuracy_score

from svae.vae import M1_M2, predict_classes_loader
from svae.training import training_M1M2

from paper_experiments.load_MNIST import make_splits_loaders_MNIST

In [ ]:
# make splits
NUM_WORKERS = os.cpu_count() // 2
train_loader, val_loader, test_loader = make_splits_loaders_MNIST(batch_size=100,
                                                                test_batch_size=100,
                                                                num_workers=NUM_WORKERS, 
                                                                pin_memory=True)

We will only test on $N = 100$ 

In [ ]:
test_loader = [batch for i, batch in enumerate(test_loader) if i < 1]

## **Configuration**

In [ ]:
base_path = "/content/drive/MyDrive/HVAE/Table2/"
os.makedirs(base_path, exist_ok=True)

In [ ]:
EPOCHS = 500
INPUT_DIM = 784
HIDDEN_DIM = 500
N_CLUSTERS = 10
LATENT_MODE = 'sample'
WARMUP = None
PATIENCE = 50
ONE_LAYER = True
LR = 1e-3

N_RUNS = 1

In [ ]:
def run_and_test(mode1, mode2, latent1, latent2, test_dataloader):
    addon = "[SVAE]" if mode == "svae" else "[NVAE]"

    print(f"\n{addon} Instantiating SVAE and optimizer..")
    model = M1_M2(mode1,
                mode2,
                INPUT_DIM,
                HIDDEN_DIM,
                latent1,
                latent2,
                N_CLUSTERS,
                ONE_LAYER,
                )
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)

    print(f"{addon} Started training..")
    model, losses, all_parts = training_M1(train_loader, 
                                    val_loader,
                                    model,
                                    optimizer,
                                    epochs=EPOCHS,
                                    beta_kl=1,
                                    warmup=WARMUP,
                                    patience=PATIENCE,
                                    show_loss_every=1,
                                    mode=LATENT_MODE)

    print(f"\n{addon} Predicting classes..")
    Y, Y_hat = predict_classes_loader(model, test_dataloader, LATENT_MODE)

    test_acc = accuracy_score(Y, Y_hat)
    print(f"{addon} Test accuracy: {test_acc*100:.2f}")
    return test_acc

In [ ]:
def launch_experiment(mode1, mode2, test_dataloader):
    results_df = {"Latent M1":[], "Latent M2":[], "Model":[], "Accuracy":[], "Std":[], "N_test":[]}
    for latent1 in tqdm([5, 10, 50], desc="Exploring latent1.."):
        for latent2 in tqdm([5, 10, 50], desc="Exploring latent2.."):
            accuracy_list = []
            for i in tqdm(list(range(N_RUNS)), desc="Repeated runs.."):
                print(f"\nSTARTING RUN n°{i+1}")
                accuracy_list.append(run_and_test(mode1, mode2, latent1, latent2, test_dataloader))
            avg_acc = np.nanmean(accuracy_list)
            std_acc = np.nanstd(accuracy_list)

            results_df["Latent M1"].append(latent1)
            results_df["Latent M2"].append(latent2)
            results_df["Model"].append(mode1 + " + " + mode2)
            results_df["Accuracy"].append(avg_acc)
            results_df["Std"].append(std_acc)
            results_df["N_test"].append(100)
    return pd.DataFrame(results_df)

## **SVAE + SVAE**

In [ ]:
results_df_svae_svae = launch_experiment("svae", "svae", test_loader)

In [ ]:
results_df_svae_svae.to_csv(base_path + "SVAE_SVAE_M1M2_results.csv")
results_df_svae_svae

## **NVAE + NVAE**

In [ ]:
results_df_nvae_nvae = launch_experiment("normal", "normal", test_loader)

In [ ]:
results_df_nvae_nvae.to_csv(base_path + "NVAE_NVAE_M1M2_results.csv")
results_df_nvae_nvae

## **SVAE + NVAE**

In [ ]:
results_df_svae_nvae = launch_experiment("svae", "normal", test_loader)

In [ ]:
results_df_svae_nvae.to_csv(base_path + "SVAE_NVAE_M1M2_results.csv")
results_df_svae_nvae

## **NVAE + SVAE**

In [ ]:
results_df_nvae_svae = launch_experiment("normal", "svae", test_loader)

In [ ]:
results_df_nvae_svae.to_csv(base_path + "NVAE_SVAE_M1M2_results.csv")
results_df_nvae_svae